In [1]:
pip install transformers==4.57.0

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
Reason for being yanked: Error in the setup causing installation issues
Using cached transformers-4.57.0-py3-none-any.whl (12.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip uninstall flash_attn -y
!pip install flash_attn==2.7.3 --no-cache

Found existing installation: flash_attn 2.8.3
Uninstalling flash_attn-2.8.3:
  Successfully uninstalled flash_attn-2.8.3
Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.9 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'flash_attn' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'flash_attn'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for flash_attn: filename=flash_attn-2.7.3-cp312-cp312-linux_x86_64.whl size=414494788 sha256=567bddcae6f7c133fd964bed9988926fe7aabaddb58bf62a744b2f782a7d4269
  Stored in director

In [3]:
pip install accelerate bitsandbytes

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached accelerate-1.11.0-py3-none-any.whl (375 kB)
Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl (60.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate]2 [accelerate]
Note: you may need to restart the kernel to use updated packages.


# Restart Kernel after installation!

In [1]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
import torch

model = Qwen3VLForConditionalGeneration.from_pretrained(
     "unsloth/Qwen3-VL-8B-Instruct-bnb-4bit",
     dtype=torch.bfloat16,
     attn_implementation="flash_attention_2",
     device_map="cuda",
)

processor = AutoProcessor.from_pretrained("unsloth/Qwen3-VL-8B-Instruct-bnb-4bit")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def generate_answer(prompt, image):
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]
    
    # Preparation for inference
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    ).to("cuda")
    
    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128, do_sample=True, temperature=0.1)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text[0]

In [4]:
generate_answer("Describe this image. Tell me a fun fact about this puppy.", "https://cf.ltkcdn.net/dogs/puppies/images/std/321597-800x515-cutest-puppy-videos.jpg")

'This is an adorable and whimsical image of a tiny puppy, likely a Chihuahua or a similar small breed, nestled snugly inside a large, white ceramic mug decorated with bold red polka dots. The puppy has a black and tan coat, with a distinctive black mask over its face and expressive, wide, dark eyes that gaze directly at the viewer. Its tiny pink paws are visible over the rim of the mug, and its tongue is slightly sticking out in a cute, playful expression. The scene is set outdoors on vibrant green artificial grass, with a white picket fence softly blurred in the background, adding to the'